### Understanding Feature Engineering -Categorical Data (Article in Towards Data Science)


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling

from mlsettings.settings import load_app_config, get_datafolder_path
 
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 
np.set_printoptions(precision=4)

pd.set_option('display.width', 200)
pd.set_option('precision', 4)
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
sns.set_style("whitegrid")
pd.options.display.float_format = '{:,.4f}'.format
sns.set()
import logging
logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
logger.setLevel(logging.DEBUG)


In [3]:
load_app_config()
DATA_DIRECTORY='vgsales'
TRAIN_FILE = 'vgsales.csv'
input_path = get_datafolder_path()

In [4]:
file_path = pathlib.Path(input_path).joinpath(DATA_DIRECTORY)
input_file = file_path.joinpath(TRAIN_FILE)

In [5]:
vg_df = pd.read_csv(input_file,encoding='utf-8')

In [6]:
#pandas_profiling.ProfileReport(vg_df)

#### Transforming Nominal Attributes
Nominal attributes consist of discrete categorical values with no notion or sense of order amongst them. The idea here is to transform these attributes into a more representative numerical format which can be easily understood by downstream code and pipelines.

In [7]:
vg_df[['Name', 'Platform', 'Year', 'Genre', 'Publisher']].head()

,Name,Platform,Year,Genre,Publisher
0,Wii Sports,Wii,"2,006.0000",Sports,Nintendo
1,Super Mario Bros.,NES,"1,985.0000",Platform,Nintendo
2,Mario Kart Wii,Wii,"2,008.0000",Racing,Nintendo
3,Wii Sports Resort,Wii,"2,009.0000",Sports,Nintendo
4,Pokemon Red/Pokemon Blue,GB,"1,996.0000",Role-Playing,Nintendo


In [8]:
vg_df['Genre'].unique()

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy'], dtype=object)

In [9]:
from sklearn.preprocessing  import LabelEncoder
label_encoder  = LabelEncoder()
genre_labels = label_encoder.fit_transform(vg_df['Genre'])
genre_mappings  = { index:label for index,label in enumerate(label_encoder.classes_)}
genre_mappings

{0: 'Action',
 1: 'Adventure',
 2: 'Fighting',
 3: 'Misc',
 4: 'Platform',
 5: 'Puzzle',
 6: 'Racing',
 7: 'Role-Playing',
 8: 'Shooter',
 9: 'Simulation',
 10: 'Sports',
 11: 'Strategy'}

In [10]:
vg_df['GenreLabel'] =genre_labels
vg_df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,GenreLabel
0,1,Wii Sports,Wii,"2,006.0000",Sports,Nintendo,41.4900,29.0200,3.7700,8.4600,82.7400,10
1,2,Super Mario Bros.,NES,"1,985.0000",Platform,Nintendo,29.0800,3.5800,6.8100,0.7700,40.2400,4
2,3,Mario Kart Wii,Wii,"2,008.0000",Racing,Nintendo,15.8500,12.8800,3.7900,3.3100,35.8200,6
3,4,Wii Sports Resort,Wii,"2,009.0000",Sports,Nintendo,15.7500,11.0100,3.2800,2.9600,33.0000,10
4,5,Pokemon Red/Pokemon Blue,GB,"1,996.0000",Role-Playing,Nintendo,11.2700,8.8900,10.2200,1.0000,31.3700,7


#### Transforming Ordinal Attributes

In [11]:
DATA_DIRECTORY2='pokemon'
TRAIN_FILE2 = 'pokemon.csv'

In [12]:
file_path = pathlib.Path(input_path).joinpath(DATA_DIRECTORY2)
input_file = file_path.joinpath(TRAIN_FILE2)

In [13]:
pokemon_df = pd.read_csv(input_file,encoding='utf-8')

In [14]:
#pandas_profiling.ProfileReport(pokemon_df)

In [15]:
gen_map = { gen:idx+1 for idx ,gen in enumerate(sorted(pokemon_df['Generation'].unique()))}

In [16]:
gen_map

{'GEN 1': 1, 'GEN 2': 2, 'GEN 3': 3, 'GEN 4': 4, 'GEN 5': 5, 'GEN 6': 6}

In [17]:
pokemon_df['Generation_label'] = pokemon_df['Generation'].map(gen_map)

In [18]:
pokemon_df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Generation_label
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,GEN 1,False,1
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,GEN 1,False,1
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,GEN 1,False,1
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,GEN 1,False,1
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,GEN 1,False,1


#### Encoding Categorical Attributes

##### For columns with text attributes

In [19]:
from sklearn import feature_extraction
def one_hot_dataframe(data,columns,replace=False):
    fe_vec= feature_extraction.DictVectorizer()
    make_dict = lambda row :dict((column,row[column]) for column in  columns)
    vector_data=pd.DataFrame(fe_vec.fit_transform(
                             data[columns].apply(make_dict, axis=1)).toarray())
    vector_data.columns = fe_vec.get_feature_names()
    vector_data.index= data.index
    if replace:
        data = data.drop(columns, axis=1)
        data = data.join(vector_data)
    return data,vector_data

In [20]:
pokemon_df,pokemon_df_ignore = one_hot_dataframe(pokemon_df, ['Generation'], replace=True)

In [21]:
pokemon_df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,Generation_label,Generation=GEN 1,Generation=GEN 2,Generation=GEN 3,Generation=GEN 4,Generation=GEN 5,Generation=GEN 6
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,False,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,False,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,False,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,False,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,False,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000


##### For columns with numeric or boolean attributes

In [22]:
gen_dummy_features = pd.get_dummies(pokemon_df['Legendary'],prefix ='Legend')

In [23]:
pokemon_df = pd.concat([pokemon_df, gen_dummy_features], axis=1)
pokemon_df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,Generation_label,Generation=GEN 1,Generation=GEN 2,Generation=GEN 3,Generation=GEN 4,Generation=GEN 5,Generation=GEN 6,Legend_False,Legend_True
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,False,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1,0
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,False,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1,0
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,False,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1,0
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,False,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1,0
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,False,1,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1,0


##### Effect Coding Scheme

The effect coding scheme is actually very similar to the dummy coding scheme,except during the encoding process, the encoded features or feature vector, for the category values which represent all 0 in the dummy coding scheme, is replaced by -1 in the effect coding scheme.

#### Curse of Dimensionality

The encoding schemes we discussed so far, work quite well on categorical data in general, but they start causing problems when the number of distinct categories in any feature becomes very large. Essential for any categorical feature of m distinct labels, you get m separate features. This can easily increase the size of the feature set causing problems like storage issues, model training problems with regard to time, space and memory. Besides this, we also have to deal with what is popularly known as the **‘curse of dimensionality’** where basically with an enormous number of features and not enough representative samples, model performance starts getting affected often leading to overfitting.

##### Bin counting scheme
The bin-counting scheme is a useful scheme for dealing with categorical variables having many categories. In this scheme, instead of using the actual label values for encoding, we use probability based statistical information about the value and the actual target or response value which we aim to predict in our modeling efforts.

##### Feature Hashing Scheme
In this scheme, a hash function is typically used with the number of encoded features pre-set (as a vector of pre-defined length) such that the hashed values of the features are used as indices in this pre-defined vector and values are updated accordingly. 

Since a hash function maps a large number of values into a small finite set of values, multiple different values might create the same hash which is termed as collisions.

Typically, a signed hash function is used so that the sign of the value obtained from the hash is used as the sign of the value which is stored in the final feature vector at the appropriate index. This should ensure lesser collisions and lesser accumulation of error due to collisions.


In [24]:
print("Total game genres:", len(vg_df['Genre'].unique()))
vg_df['Genre'].unique()

Total game genres: 12


array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy'], dtype=object)

In [25]:
from sklearn.feature_extraction import FeatureHasher
fh = FeatureHasher(n_features=6, input_type='string')
hashed_features = fh.fit_transform(vg_df['Genre'])
hashed_features = hashed_features.toarray()
hashed_features = pd.DataFrame(hashed_features)
hashed_features = hashed_features.add_prefix('Genre') 
vg_df = pd.concat([vg_df,hashed_features], axis=1)
vg_df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,GenreLabel,Genre0,Genre1,Genre2,Genre3,Genre4,Genre5
0,1,Wii Sports,Wii,"2,006.0000",Sports,Nintendo,41.4900,29.0200,3.7700,8.4600,82.7400,10,-2.0000,2.0000,0.0000,-2.0000,0.0000,0.0000
1,2,Super Mario Bros.,NES,"1,985.0000",Platform,Nintendo,29.0800,3.5800,6.8100,0.7700,40.2400,4,0.0000,2.0000,2.0000,-1.0000,1.0000,0.0000
2,3,Mario Kart Wii,Wii,"2,008.0000",Racing,Nintendo,15.8500,12.8800,3.7900,3.3100,35.8200,6,-1.0000,0.0000,0.0000,0.0000,0.0000,-1.0000
3,4,Wii Sports Resort,Wii,"2,009.0000",Sports,Nintendo,15.7500,11.0100,3.2800,2.9600,33.0000,10,-2.0000,2.0000,0.0000,-2.0000,0.0000,0.0000
4,5,Pokemon Red/Pokemon Blue,GB,"1,996.0000",Role-Playing,Nintendo,11.2700,8.8900,10.2200,1.0000,31.3700,7,-1.0000,1.0000,2.0000,0.0000,1.0000,-1.0000


In [26]:
pandas_profiling.ProfileReport(vg_df)

Number of variables,18
Number of observations,16598
Total Missing (%),0.1%
Total size in memory,2.2 MiB
Average record size in memory,140.0 B
Numeric,12
Categorical,4
Boolean,1
Date,0
Text (Unique),0
Rejected,1
